# **Intrusion Dection**

The purpose of this project is to train machine learning models that can classify streams and be able to recognize different attacks, i.e., models used for multi-class classification tasks.<br>
The NSL-KDD dataset (KDDTrainClean.csv) represents a benchmark for Intrusion Detection systems. The dataset contains information about network traffic flows to an IT infrastructure. Each flow is labeled as “normal” or associated with an attack type. The label column of the dataset represents the label.

In [11]:
import os
import logging
import math
import shutil
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow as tf

from tensorflow import keras
from keras_tuner import RandomSearch
from keras import layers, models
from keras.utils import plot_model

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from scipy.stats import uniform
from rich.console import Console
from rich.table import Table
from rich.text import Text

 

2025-01-12 18:06:47.318565: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-12 18:06:47.324214: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-12 18:06:47.386255: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-12 18:06:47.449103: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736701607.499687   23922 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736701607.51

In [12]:
#Ignore the warning
warnings.filterwarnings("ignore")

#Removes informational logs and warnings of TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  
tf.get_logger().setLevel(logging.ERROR)

os.environ['PYTHONWARNINGS'] = 'ignore'

logging.getLogger('tensorflow').setLevel(logging.ERROR)

#Clear keras tuner directory
shutil.rmtree('tuning/', ignore_errors=True)

console = Console()

# Funzione per stampare una tabella
def print_table(title, columns, rows):
    table = Table(title=title)
    for col in columns:
        table.add_column(col['header'], justify=col.get('justify', 'center'), style=col.get('style', 'bright_white'))
    for row in rows:
        table.add_row(*row)
    console.print(table)

# Funzione per stampare una tabella di valutazione del modello
def print_evaluation_table(model_name, test_class_report, test_accuracy):
    rows = [
        ["Accuracy", f"{test_accuracy:.4f}"],
        ["Precision (wgtd_avg)", f"{test_class_report['weighted avg']['precision']:.4f}"],
        ["Recall (wgtd_avg)", f"{test_class_report['weighted avg']['recall']:.4f}"],
        ["F1-Score (wgtd_avg)", f"{test_class_report['weighted avg']['f1-score']:.4f}"]
    ]
    columns = [{"header": "Metric", "style": "bright_cyan"}, {"header": "Result", "style": "bright_green", "justify": "center"}]
    print_table("Model Evaluation Results for " + model_name, columns, rows)

# Funzione per stampare una tabella con i migliori iperparametri
def print_best_params_table(model_name, best_params):
    rows = [[param, str(value)] for param, value in best_params.items()]
    columns = [{"header": "Hyperparameter", "style": "bright_cyan"}, {"header": "Value", "style": "bright_magenta", "justify": "center"}]
    print_table("Best Hyperparameters for " + model_name, columns, rows)

In [ ]:
file_path = "./KDDTrainClean.csv"
data = pd.read_csv(file_path)
#data.head()


Number of raw of the dataset
125973
